## Chapter 6: Seismic data

Plotting seismic data in Python is a straightforward process. Once loaded, the data typically takes the form of a NumPy array containing amplitude values. In this example, we’ll work with a subset of the [F3 seismic dataset](https://terranubis.com/datainfo/F3-Demo-2020)—a 3D seismic
cube acquired offshore in the Netherlands.

The `segy` file containing the seismic data is relatively large (370 MB), so it has been hosted on a remote server. To download it, we can use Python’s `urllib` library. The process is straightforward: define the URL of the file and the local filename, then pass these to the `urlretrieve()` function to download and save the file in the desired directory.

In [ ]:
# import libraries required for the notebook
import os # import os to work with directories
from urllib import request # for downloading files
import numpy as np # import numpy as np
import matplotlib.pyplot as plt # import matplotlib.pyplot as plt
from ipywidgets import interact # import ipywidgets interact
import plotly.graph_objects as go # import plotly.graph_objects as go
from plotly.subplots import make_subplots # from plotly.subplots import make_subplots
import segyio # import segyio for reading SEG-Y files
import plot_utilities as pu # import our plot utilities package

In [ ]:
# retrieve the data file from a remote server

# Define the remote file to retrieve
remote_url = "http://www.ux.uis.no/~nestor/Public/f3-dsmf.sgy"
# Define the local filename to save data
local_file = os.path.join("..", "data", "f3-dsmf.sgy")
# Download remote and save locally
request.urlretrieve(remote_url, local_file)

To read the `segy` data, we’ll use the [Segyio](https://github.com/equinor/segyio) library, a Python package designed for efficient handling of SEG-Yfiles. We can open the file using the Segyio `open()` method. Once opened, we can access the inline, crossline, and two-way
travel time (TWT) axes using the `ilines`, `xlines`, and `samples` attributes of the file object `f`, respectively. The range of each axis can be calculated by subtracting the minimum value from the maximum, and the number of elements in each can be obtained from the shape of the corresponding array.

In [ ]:
# print inline, xline and twt ranges

filename = os.path.join("..", "data", "f3-dsmf.sgy") # segy file

# open segy file and print inline, xline and twt ranges
with segyio.open(filename,"r") as f:
    s = [f.ilines, f.xlines, f.samples]
    s_t = ["IL", "XL", "TWT"]
    for i in range(len(s)):
        rg = np.array([np.amin(s[i]), np.amax(s[i])])
        count = s[i].shape[0]
        print(f"{s_t[i]} range: {rg}, count: {count}")

The seismic data spans inlines from 200 to 600, xlines from 500 to 1100, and TWT from 200 to 1500 ms. We can also determine the sampling intervals—that is, the step size between adjacent inlines, xlines, and time samples—to better understand the resolution of the dataset.

In [ ]:
# inline step
il_st = (np.amax(f.ilines) 
         - np.amin(f.ilines)) / (f.ilines.shape[0] - 1)
# xline step 
xl_st = (np.amax(f.xlines) 
         - np.amin(f.xlines)) / (f.xlines.shape[0] - 1)
# twt step 
twt_st = (np.amax(f.samples) 
          - np.amin(f.samples)) / (f.samples.shape[0] - 1)
# print steps
print(f"IL/XL/TWT steps: {il_st}, {xl_st}, {twt_st}") 

In this dataset, the inline and xline increments are 1, while the TWT increment is 4 ms. To read the seismic cube, we can use the Segyio `tools.cube()` method, passing the file name as input. After loading the data, we print the shape of the resulting NumPy array to confirm its dimensions along the inline, xline, and TWT axes.

In [ ]:
# read seismic data
data = segyio.tools.cube(filename)

# maximum amplitude value for plotting
vmax = np.percentile(np.abs(data), 99) 

# cube shape
shape = data.shape
print(f"{shape[0]} IL, {shape[1]} XL and {shape[2]} TWT samples") 

In the code above, `vmax` is set to the 99th percentile of the absolute amplitude values. This value will be used later to scale the color range when plotting the data. Since we’re working with a subset of the original 3D cube, the final step before visualization is to assign the correct inline, xline, and TWT values to the axis ticks. This is done by creating arrays of tick positions and corresponding labels for each axis.

In [ ]:
il_tl = 100 # step between tick labels for inlines
xl_tl = 100 # step between tick labels for xlines
twt_tl = 200 # step between tick labels for twt

# inlines ticks positions and labels
il_pos = np.arange(0, shape[0], il_tl/il_st) 
il_lab = il_pos * il_st + f.ilines[0] 
il_lab = il_lab.astype(int) 
print(f"il ticks = {il_pos}\n and labels = {il_lab}")

# xlines ticks positions and labels
xl_pos = np.arange(0, shape[1], xl_tl/xl_st) 
xl_lab = xl_pos * xl_st + f.xlines[0] 
xl_lab = xl_lab.astype(int) 
print(f"xl ticks = {xl_pos}\n and labels = {xl_lab}")

# TWT ticks positions and labels
twt_pos = np.arange(0, shape[2], twt_tl/twt_st) 
twt_lab = twt_pos * twt_st + f.samples[0] 
twt_lab = twt_lab.astype(int) 
print(f"twt ticks = {twt_pos}\n and labels = {twt_lab}")

We can now plot the data. To facilitate this process, I have made two functions in the [utilities](plot_utilities/utilities.py) module of our plot_utilities package:

- Function `plot_trace` plots a trace.

- Function `plot_slice` plots a slice, which can be either an inline, xline, or time slice. In this function, we use the Matplotlib `imshow()` method to visualize the seismic data as a pseudocolor image.

Let’s start by plotting one trace of the seismic data at inline 400 and xline 800:

In [ ]:
# plot trace

il, xl = 400, 800 # inline and xline

il_id = int((il - f.ilines[0]) / il_st) # inline index

xl_id = int((xl - f.xlines[0]) / xl_st) # xline index

trace = data[il_id, xl_id, :] # get trace data

time_id = np.arange(shape[2]) # time index

# create figure
fig, ax = plt.subplots(figsize=(3, 7)) # 1 subplot

# plot trace
pu.plot_trace(trace, time_id, vmax, twt_pos, twt_lab, ax)

Now, let´s plot a slice, for example inline 350:

In [ ]:
# plot slice

# slice type: inline, xline or time
sl_type = "inline" 

# slice value: allowed values depend on the slice type
# for inline the value should be between 200 and 600,
# for xline between 500 and 1100,
# and for time between 200 and 1500 ms
value = 350 

# slice and axes ticks positions and labels
if sl_type == "inline":
    index = int((value - f.ilines[0]) / il_st) # inline index
    slice = data[index, :, :]
    ax_pos = [xl_pos, twt_pos]
    ax_lab = [xl_lab, twt_lab]
    title = f"{sl_type} {value}" 
elif sl_type == "xline":
    index = int((value - f.xlines[0]) / xl_st)
    slice = data[:, index, :] 
    ax_pos = [il_pos, twt_pos]
    ax_lab = [il_lab, twt_lab]
    title = f"{sl_type} {value}"
elif sl_type == "time":
    index = int((value - f.samples[0]) / twt_st)
    slice = data[:, :, index] 
    ax_pos = [xl_pos, il_pos]
    ax_lab = [xl_lab, il_lab]
    title = f"{sl_type} {value} ms"

# create figure
fig, ax = plt.subplots(figsize=(8, 6)) # 1 subplot

# plot slice
pu.plot_slice(slice, vmax, title, sl_type, 
              ax_pos, ax_lab, fig, ax, cb=False)

plt.show() # show plot

Note that the code above is quite flexible. You can change the slice type (`sl_type`) and/or the slice value (`value`) to visualize another slice. For example, try visualizing xline 700, and time slice 1000.

Next, we’ll use [ipywidgets](https://github.com/jupyter-widgets/ipywidgets) to interactively explore the data, allowing us to change the inline, xline and time slices dynamically. ipywidgets, also referred to as Jupyter widgets or simply widgets, are interactive HTML elements designed for use in Jupyter notebooks. By defining a plotting function and associating it with a widget, we can dynamically adjust the slices using slider buttons:

In [ ]:
def update_plot(il, xl, time):
    """
    updates the plot with the selcted inline, xline
    and time values
    """
    # indexes for inline, xline and time
    il_id = int ((il - f.ilines[0]) / il_st) 
    xl_id = int ((xl - f.xlines[0]) / xl_st)
    time_id = int ((time - f.samples[0]) / twt_st) 

    # create figure
    fig, ax = plt.subplots(2,2,figsize=(14,12))

    # lists to facilitate plotting
    sl_types = ["inline", "xline", "time"]
    slices = [data[il_id, :, :], data[:, xl_id, :], 
              data[:, :, time_id]]
    ax_pos = [[xl_pos, twt_pos], [il_pos, twt_pos], 
              [xl_pos, il_pos]]
    ax_lab = [[xl_lab, twt_lab], [il_lab, twt_lab], 
              [xl_lab, il_lab]]
    titles = [f"Inline {il:.0f}", f"Xline {xl:.0f}", 
              f"Time {time:.0f} ms"]
    lines = [[xl_id, time_id], [il_id, time_id], 
             [xl_id, il_id]]

    # plot slices
    for i, sl_type in enumerate(sl_types):
        # plot slice
        pu.plot_slice(slices[i], vmax, titles[i], sl_type, 
                      ax_pos[i], ax_lab[i], fig, 
                      ax[i//2, i%2], cb=False)       
        # plot lines 
        ax[i//2, i%2].axvline(lines[i][0], color='k', 
                              linestyle='--', linewidth=2)
        ax[i//2, i%2].axhline(lines[i][1], color='k', 
                              linestyle='--', linewidth=2)

    # four subplot is not used
    ax[1, 1].axis('off')

    # present figure
    fig.tight_layout()
    plt.show()

# interact with the function
interact(update_plot, 
         il=(np.amin(f.ilines), np.amax(f.ilines), il_st), 
         xl=(np.amin(f.xlines), np.amax(f.xlines), xl_st), 
         time=(np.amin(f.samples), np.amax(f.samples), twt_st));

The first slider allows us to select the inline, the second slider the xline, and the third slider the time slice. The black dashed lines show the location of the slices.

## Interactive plots

ipywidgets allow us to dynamically adjust the input parameters of a plot, but the plot itself remains static — moving the cursor over it doesn’t reveal any information, and we cannot zoom or pan. To add this kind of interactivity, we can use the [Plotly](https://plotly.com/python/) library. Plotly offers extensive capabilities for creating interactive 2D and 3D plots, as well as maps. In this section, we will provide a brief introduction to its features.

Let’s begin by plotting a trace from our seismic cube. For this, we use the Plotly `graph_objects` (`go`) module:

In [ ]:
fig = go.Figure() # create figure

# convert time index to time values
time = time_id * twt_st + f.samples[0] 

# add trace to the figure
fig.add_trace(go.Scatter(x=trace, y=time, 
                         mode="lines", 
                         line=dict(color="black")))

# set x and y axis
fig.update_xaxes(range=[-vmax, vmax], 
                 title_text="Amplitude")
fig.update_yaxes(range=[time[-1], time[0]], 
                 title_text="Time [ms]")

# set figure size
fig.update_layout(width=300, height=600)

fig.show() # show the figure

Notice that as you hover the cursor over the plot, the amplitude and time values are displayed.

Now, let’s plot the trace alongside the time slice. To create two subplots— one for the trace and another for the time slice — we need to use the `make_subplots()` method from the Plotly subplots module:

In [ ]:
# Extract a time slice
value = 1200
index = int((value - f.samples[0]) / twt_st) # time index
slice = data[:, :, index]

# Create subplots
fig = make_subplots(rows=1, cols=2, 
                    subplot_titles=(None, f"Time = {value} ms"))

# elements to draw
el_defs = [
    # trace on 1st subplot
    (go.Scatter(x=trace, y=time, mode="lines", name="Trace", 
                line_color="black"), 1, 1),
    # slice on 2nd subplot
    (go.Heatmap(z=slice, colorscale="RdBu", name="Amplitude", 
                showscale=False, zmin=-vmax, zmax=vmax), 1, 2),
    # trace location on 2nd subplot
    (go.Scatter(x=[xl_id], y=[il_id], mode="markers", name="Trace", 
                marker=dict(size=10, color="black")), 1, 2)
]
# add to the figure
for el, row, col in el_defs:
    fig.add_trace(el, row=row, col=col)

# update axes
fig.update_xaxes(title_text="Amplitude", 
                 row=1, col=1, range=[-vmax, vmax])
fig.update_yaxes(title_text="Time [ms]", 
                 row=1, col=1, autorange="reversed")
xl_text = [str(v) for v in xl_lab]
fig.update_xaxes(title_text="Xline", tickvals=xl_pos, 
                 ticktext=xl_text, row=1, col=2)
il_text = [str(v) for v in il_lab]
fig.update_yaxes(title_text="Inline", tickvals = il_pos,
                 ticktext=il_text, row=1, col=2)

# figure size and legend off
fig.update_layout(width=900, height=600, showlegend=False)

fig.show()

We have only begun to explore what Plotly offers. With a bit of practice, you’ll find it a powerful tool for creating clear and interactive visualizations. I encourage you to experiment further and discover how it can support your own projects.